In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# should import EXIOBASE database before
# and also set biosphere, LCIA methods
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 2 object(s):
	EXIOBASE 3.3.17 hybrid
	biosphere3

In [4]:
len(bw.methods)

850

In [5]:
eidb = bw.Database('EXIOBASE 3.3.17 hybrid')
len(eidb)

7872

In [11]:
import random

# randomly pick 20 methods from the bw.methods
methods = random.sample(list(bw.methods), 20)
methods

[('ReCiPe Midpoint (H) V1.13 no LT', 'ozone depletion', 'ODPinf'),
 ('EDIP2003 w/o LT (obsolete)',
  'non-renewable resources w/o LT',
  'cadmium w/o LT'),
 ('ReCiPe Endpoint (I,A) (obsolete)', 'resources', 'total'),
 ('EDIP (obsolete)', 'resource consumption', 'non-renewable resources, iron'),
 ('CML 2001 (obsolete)', 'climate change', 'upper limit of net GWP'),
 ('ReCiPe Endpoint (E,A) w/o LT (obsolete)',
  'human health w/o LT',
  'human toxicity w/o LT'),
 ('ReCiPe Endpoint (E,A) (obsolete)',
  'ecosystem quality',
  'marine ecotoxicity'),
 ('cumulative exergy demand',
  'solar',
  'renewable energy resources, solar, converted'),
 ('CML 2001 w/o LT (obsolete)',
  'marine aquatic ecotoxicity w/o LT',
  'MAETP infinite w/o LT'),
 ('ReCiPe Endpoint (H,A) (obsolete)',
  'human health',
  'photochemical oxidant formation'),
 ('ecological scarcity 2013 no LT', 'heavy metals into water', 'total'),
 ('ReCiPe Endpoint (I,A) (obsolete)',
  'human health',
  'particulate matter formation'),
 

In [12]:
random_act = eidb.random()
random_act.as_dict()

{'location': 'RO',
 'name': 'Re-processing of secondary steel into new steel',
 'code 1': 'i27.a.w',
 'code 2': 'A_STEW',
 'type': 'process',
 'format': 'Exiobase 3.3.17 hybrid mrio_common_metadata tidy datapackage',
 'key': ('EXIOBASE 3.3.17 hybrid',
  'RO-Re-processing of secondary steel into new steel'),
 'unit': 'ton',
 'database': 'EXIOBASE 3.3.17 hybrid',
 'code': 'RO-Re-processing of secondary steel into new steel'}

In [13]:
lca = bw.LCA({random_act:1}, methods[0])
lca.lci()
lca.lcia()
lca.score

0.0

In [14]:
len(lca.biosphere_dict) # number of bio flows

58

In [15]:
results_array = np.zeros((len(eidb), len(methods))) # to save the results

In [16]:
method_matrices = []

for m in methods:
    lca.switch_method(m)
    method_matrices.append(lca.characterization_matrix.copy())

In [19]:
import pyprind

for i, ds in enumerate(pyprind.prog_bar(eidb)):
    lca.redo_lci({ds: 1})
    for j, method in enumerate(methods):
        vector = np.ravel((method_matrices[j] * lca.inventory).sum(axis=1))
        if not vector.sum():
            continue
        results_array[i, j] = vector.sum()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:19:56


In [1]:
# also tried to just use lca.score but didn't work as there were a lot of zero values
# for i, ds in enumerate(pyprind.prog_bar(eidb)):
#    lca.redo_lci({ds: 1})
#    for j, method in enumerate(methods):
#        if not lca.score:
#            continue
#        results_array[i, j] = lca.score

In [29]:
# get the information of all activities like name, location
act_dict = {}
act_id = [] # activity id is the number from the lca.activity_dict, to represent the activity
act_name = []
act_loc = []
act_type = []
act_unit = []
act_code = []
for obj in eidb:
    act_id.append(lca.activity_dict[obj['key']])
    act_name.append(obj['name'])
    act_loc.append(obj['location'])
    act_type.append(obj['type'])
    act_unit.append(obj['unit'])
    act_code.append(obj['code'])
len(act_id),len(act_name),len(act_loc),len(act_type),len(act_unit),len(act_code) # check the number of each item

(7872, 7872, 7872, 7872, 7872, 7872)

In [30]:
# make the information as a date frame
act_dict['activity'] = act_id
act_dict['name'] = act_name
act_dict['location'] = act_loc
act_dict['type'] = act_type
act_dict['unit'] = act_unit
act_dict['code'] = act_code
df = pd.DataFrame(act_dict)
df.head()

,activity,name,location,type,unit,code
0,522,Mining of other non-ferrous metal ores and con...,BR,process,ton,BR-Mining of other non-ferrous metal ores and ...
1,3936,Cultivation of paddy rice,LU,process,ton,LU-Cultivation of paddy rice
2,1143,Activities of membership organisation n.e.c.,CN,process,Meuro,CN-Activities of membership organisation n.e.c...
3,3666,"Plastics, basic",LV,process,ton,"LV-Plastics, basic"
4,4956,Processing of meat poultry,PT,process,ton,PT-Processing of meat poultry


In [31]:
df.to_csv('exiobase_info_lcia.csv')# save file

In [32]:
bio = bw.Database('biosphere3')
random_bio = bio.random()
random_bio.as_dict()

{'categories': ('natural resource', 'land'),
 'code': '9197fa8a-1f9a-427b-91d3-113260ae6eb1',
 'name': 'Transformation, to seabed, natural (non-use)',
 'database': 'biosphere3',
 'unit': 'square meter',
 'type': 'natural resource'}

In [33]:
# get reversed lca.biosphere_dict
rev_bio_dict = {value:key for key, value in lca.biosphere_dict.items()}
len(rev_bio_dict)

58

In [34]:
# get the information of bio flows
bio_dict = {}
bio_id = [] # a number from lca.biosphere_dict to represent the bio flow
bio_category = []
bio_name = []
bio_unit = []
bio_type = []
for obj in bio:
    for n in range(58):
        if obj.key == rev_bio_dict[n]:
            bio_id.append(n)
            bio_name.append(obj['name'])
            bio_category.append(obj['categories'])
            bio_unit.append(obj['unit'])
            bio_type.append(obj['type'])
            break
len(bio_id),len(bio_category),len(bio_name),len(bio_unit),len(bio_type) # check the number of each item

(58, 58, 58, 58, 58)

In [35]:
# make the info as a dataframe
bio_dict['bio_id'] = bio_id
bio_dict['name'] = bio_name
bio_dict['category'] = bio_category
bio_dict['unit'] = bio_unit
bio_dict['type'] = bio_type
df_bio = pd.DataFrame(bio_dict)
df_bio.head()

,bio_id,name,category,unit,type
0,48,"Phosphorus, in ground","(natural resource, in ground)",kilogram,natural resource
1,3,"Coal, hard, unspecified, in ground","(natural resource, in ground)",kilogram,natural resource
2,8,"Occupation, arable land, unspecified use","(natural resource, land)",square meter-year,natural resource
3,51,"Uranium, in ground","(natural resource, in ground)",kilogram,natural resource
4,45,"Lead, in ground","(natural resource, in ground)",kilogram,natural resource


In [36]:
df_bio.to_csv('exiobase_bioflows_info_lcia.csv')#save the file

In [37]:
vectors = {}

In [39]:
# get the lca.inventory scores for each bio flow of activities
from pyprind import prog_bar
n = 0
for ds in prog_bar(eidb):
    key = lca.activity_dict[ds.key]
    vectors[key] = results_array[n]
    n = n+1

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [40]:
# make the stored scores as a dataframe
df_data = pd.DataFrame(vectors)
df_data = df_data.T
df_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
3134,0.0,0.0,11201.326492,12883.777118,255829.178178,1757.368486,8.888608,0.0,456042.371358,7.124922,0.0,1783.370208,12.994451,21048.056235,2.222093e+07,0.0,3617.065065,347.679984,29.461488,564.839510
5011,0.0,0.0,141.389312,5.093702,599.315514,1.248135,0.006260,0.0,328.709211,0.004594,0.0,2.336660,0.014760,112.249001,2.355901e+04,0.0,4.561716,0.398248,0.010772,0.168818
2309,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000
1754,0.0,0.0,268.072880,37.382050,6428.640210,7.854999,0.070190,0.0,3121.043304,0.881202,0.0,19.762819,0.041784,94.142458,1.339083e+05,0.0,64.562019,4.765737,0.167417,2.421799
7229,0.0,0.0,-38.662620,0.483713,1738.408632,0.327966,0.031941,0.0,698.350954,0.005954,0.0,8.437970,0.001601,8.777430,1.689036e+04,0.0,13.617743,3.321126,-0.001926,-3.406599


In [41]:
df_data.to_csv('exiobase_lica_scores.csv') # save the file